In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='utf-8')

In [4]:
texts = df.review.tolist()
sentiment = df.sentiment.tolist()

In [5]:
amount = 1000
X_train, X_test, y_train, y_test = train_test_split(texts[:amount], sentiment[:amount])

In [6]:
# clean text
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def word_tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')#fix
    clean = tokenizer.tokenize(text)
    return clean
   
def stop_words(tokenize_words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in tokenize_words:
        if w not in stopWords:
            wordsFiltered.append(w.lower())
    return wordsFiltered


def stem(text_arr):
    ps = PorterStemmer()
    stem_arr = []
    for word in text_arr:
        stem_arr.append(ps.stem(word))
    return stem_arr

def preprocess_text(text):
    #clean html 
    clean = cleanhtml(text)
    
    # tokenize and remove punctuation 
    tokenize = word_tokenize(clean)
    
    # stop words
    stop = stop_words(tokenize)
    
    # stem 
    stem_arr = stem(stop)
    
    return stem_arr


def preprocess_texts(texts):
    tmp_set = set()
    for text in texts:
        arr = preprocess_text(text) 
        tmp_set.update(arr)
    return tmp_set

print(preprocess_text(df.to_dict('records')[0]['review']))

['with', 'stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari', 'watch', 'the', 'wiz', 'watch', 'moonwalk', 'mayb', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema', 'origin', 'releas', 'some', 'subtl', 'messag', 'mj', 'feel', 'toward', 'press', 'also', 'obviou', 'messag', 'drug', 'bad', 'kay', 'visual', 'impress', 'cours', 'michael', 'jackson', 'unless', 'remot', 'like', 'mj', 'anyway', 'go', 'hate', 'find', 'bore', 'some', 'may', 'call', 'mj', 'egotist', 'consent', 'make', 'movi', 'but', 'mj', 'fan', 'would', 'say', 'made', 'fan', 'true', 'realli', 'nice', 'the', 'actual', 'featur', 'film', 'bit', 'final', 'start', '20', 'minut', 'exclud', 'smooth', 'crimin', 'sequenc', 'joe', 'pesci', 'convinc', 'psychopath', 'power', 'drug', 'lord', 'whi', 'want', 'mj', 'dead', 'bad', 'be

In [7]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# print(train_texts)
corpus = X_train

vocab_tf = preprocess_texts(corpus)

tfidf = TfidfVectorizer(stop_words = 'english', vocabulary=vocab_tf)
X = tfidf.fit_transform(corpus)
X_dense = X.todense()

# print(tfidf.get_feature_names())
X_test_transform = tfidf.transform(X_test)
X_test_transform_dense = X_test_transform.todense()

In [8]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import numpy as np

n = range(100, 250, 10)

temp = []

clf = LogisticRegression(random_state=0).fit(X_dense, y_train)
pred = clf.predict(X_test_transform_dense)
print(classification_report(pred, y_test[:amount]))

for n_components in n:
    pca = PCA(n_components=n_components)
    pca.fit(X_dense)
    ans = pca.transform(X_dense)
    
    pca.fit(X_test_transform_dense)
    ans_test = pca.transform(X_test_transform_dense)
    
    
    clf = LogisticRegression(random_state=0).fit(ans, y_train)
    pred = clf.predict(ans_test)
    report = classification_report(pred, y_test[:amount], output_dict=True)

    print(f'n clusters = {n_components}, aaccuracy = {report["accuracy"]}')

              precision    recall  f1-score   support

           0       0.69      0.83      0.76       120
           1       0.81      0.66      0.73       130

    accuracy                           0.74       250
   macro avg       0.75      0.75      0.74       250
weighted avg       0.76      0.74      0.74       250

n clusters = 100, aaccuracy = 0.568
n clusters = 110, aaccuracy = 0.54
n clusters = 120, aaccuracy = 0.512
n clusters = 130, aaccuracy = 0.512
n clusters = 140, aaccuracy = 0.54
n clusters = 150, aaccuracy = 0.528
n clusters = 160, aaccuracy = 0.544
n clusters = 170, aaccuracy = 0.524
n clusters = 180, aaccuracy = 0.572
n clusters = 190, aaccuracy = 0.568
n clusters = 200, aaccuracy = 0.56
n clusters = 210, aaccuracy = 0.536
n clusters = 220, aaccuracy = 0.504
n clusters = 230, aaccuracy = 0.516
n clusters = 240, aaccuracy = 0.524


In [9]:
import gensim
from bs4 import BeautifulSoup
import re, string
clean = [ ]

for doc in texts:
    x = doc.lower()                     
    x = BeautifulSoup(x, 'lxml').text   
    x = re.sub('[^A-Za-z0-9]+', ' ', x)
    x = x.split(' ')
    clean.append(x)

In [10]:
print(clean[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 

In [11]:
model = gensim.models.Word2Vec(sentences=clean, vector_size=200, window=10, min_count=10, negative=15)

In [12]:
market = model.wv.most_similar('market')
print("market", market)
print()

street = model.wv.most_similar('street')
print("street", street)
print()

game = model.wv.most_similar('game')
print("game", game)
print()

lesson = model.wv.most_similar('lesson')
print("lesson", lesson)
print()

market [('studios', 0.6523382663726807), ('usa', 0.6327494978904724), ('access', 0.5971789360046387), ('columbia', 0.59209144115448), ('blockbuster', 0.5912190079689026), ('company', 0.5869526267051697), ('stores', 0.5834096670150757), ('cinemas', 0.5787006616592407), ('distributor', 0.5743589997291565), ('national', 0.573961079120636)]

street [('pickup', 0.7195087671279907), ('park', 0.7136522531509399), ('elm', 0.7083839178085327), ('streets', 0.6889354586601257), ('riding', 0.6753363609313965), ('beach', 0.6732529997825623), ('horseback', 0.6650832295417786), ('roof', 0.6633862853050232), ('mountain', 0.6598338484764099), ('nightmare', 0.6591573357582092)]

game [('games', 0.6758685111999512), ('baseball', 0.5117459297180176), ('football', 0.4533810615539551), ('basketball', 0.4491329491138458), ('mouse', 0.444535493850708), ('64', 0.4358806312084198), ('missions', 0.41914787888526917), ('store', 0.40829816460609436), ('nintendo', 0.4081835150718689), ('clip', 0.40179628133773804)]

In [13]:
from sklearn.neighbors import KDTree
print(X_dense[:10])
tree = KDTree(X_dense)
dist, ind = tree.query(X_dense[:10], k=3) 
print(dist)
print(ind)
    

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.11379233]]
[[0.         1.05969149 1.07309676]
 [0.         1.31334844 1.31999004]
 [0.         1.2416603  1.33262491]
 [0.         1.25635501 1.28977382]
 [0.         1.30840454 1.31359879]
 [0.         1.28927059 1.29435016]
 [0.         1.33713325 1.35164183]
 [0.         1.31259271 1.31421363]
 [0.         1.30682287 1.31005683]
 [0.         1.07730653 1.09457385]]
[[  0 481 474]
 [  1 253 303]
 [  2  89 128]
 [  3 676 637]
 [  4 531 331]
 [  5 645 390]
 [  6  46 566]
 [  7 503 670]
 [  8 303 108]
 [  9 297 485]]


In [15]:
def preprocessing(corpus):
    # initialize
    clean_text = []

    for row in corpus:
        # tokenize
        tokens = nltk.tokenize.word_tokenize(row)
        # lowercase
        tokens = [token.lower() for token in tokens]
        # isword
        tokens = [token for token in tokens if token.isalpha()]
        clean_sentence = ''
        clean_sentence = ' '.join(token for token in tokens)
        clean_text.append(clean_sentence)
        
    return clean_text
    
all_text = preprocessing(texts)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts
cv = CountVectorizer(ngram_range=(1,1), stop_words = 'english')
# matrix of token counts
X = cv.fit_transform(all_text)
Xc = (X.T * X) # matrix manipulation
Xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1

In [17]:
names = cv.get_feature_names()
print(names.index('market'))

37953


In [19]:
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(100)
Xpca = clf.fit_transform(Xc)
pca = PCA(n_components=100)
pca.fit(Xpca)
ans = pca.transform(Xpca)

In [20]:
print(len(ans[0]))

100


In [21]:
tree = KDTree(ans)

In [22]:
market = names.index('market')
dist, ind = tree.query([ans[market]], k=5) 
print(ind)
for i in ind:
    for j in i:
        print(names[j])

[[37953 46630 67941 48588 51353]]
market
placed
welcome
provided
reputation


In [23]:
street = names.index('street')
dist, ind = tree.query([ans[street]], k=5) 
print(ind)
for i in ind:
    for j in i:
        print(names[j])

[[59429  8267 18064 51947 23115]]
street
business
dream
rich
following


In [24]:
game = names.index('game')
dist, ind = tree.query([ans[game]], k=5) 
print(ind)
for i in ind:
    for j in i:
        print(names[j])

[[24365 22279 27170 52991 68735]]
game
fight
hand
run
wo


In [25]:
lesson = names.index('lesson')
dist, ind = tree.query([ans[lesson]], k=5) 
print(ind)
for i in ind:
    for j in i:
        print(names[j])

[[35526 29019 45126 60507 50390]]
lesson
hopes
passed
survive
record
